# ノード数のスケジュール設定を削除する
---

workerノード数のスケジュール設定を削除します。

## パラメータの指定

スケジュール設定を削除するのに必要となるパラメータを指定します。

### UnitGroup名

対象となるCoursewareHubのUnitGroup名を指定します。

UnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

#### チェック

指定されたグループ名が妥当なものであることをチェックします。

指定されたUnitGroupで、スケジュール設定が行われていることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if gvars.get("vcnode_schedule") is None:
    raise RuntimeError("スケジュール設定が行われていません")

managerノードにアクセスできることを確認します。

In [ ]:
target_group = f'{ugroup_name}_manager'
!ansible {target_group} -m ping

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2025/04/01 00:00:00 UTC: VCPAuthException: xxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## スケジュール設定の削除

### タイマーの停止

VCノードのスケールアウト、スケールインを行うsystemdタイマーを停止して無効化します。

現在のsystemdタイマーの設定状況を確認します。

In [ ]:
!ansible {target_group} -b -a 'systemctl list-timers --all cwh-worker-node.timer'

タイマー設定の停止と無効化を行います。

In [ ]:
!ansible {target_group} -b -m systemd -a \
    'name=cwh-worker-node.timer enabled=false state=stopped'

systemdタイマーの設定を表示します。`cwh-worker-node.timer` が表示されなくなっていることを確認してください。

In [ ]:
!ansible {target_group} -b -a 'systemctl list-timers --all cwh-worker-node.timer'

### ノード構成の変更

workerノード数をスケジュール定義のデフォルト設定の構成となるようにします。

スケジュール定義のデフォルト設定のノード数を確認します。

In [ ]:
print(gvars["vcnode_schedule"]["default"]["node_count"])

次のセルを実行すると、workerノード数をスケジュール設定のデフォルトのノード数と一致するような操作を行います。スケジュール設定の削除のみ行いworkerノードの構成は現在の状態をそのまま保持する場合は、次のセルを実行せずに次の節「[2.3 group_varsの更新](#group_varsの更新)」に進んでください。

In [ ]:
import json
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -Dv -l {target_group} -e @{str(vars_path)} playbooks/restore-default-nodes.yml

### group_varsの更新
　
group_varsからスケジュールに関するパラメータを削除します。またworkerノード数を現在のノード数に更新します。

現在のworkerノード数を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ut = ug.get_unit("worker")
worker_nodes = len(ut.find_nodes())
print(worker_nodes)

現在のworkerノードの状態を確認します。

In [ ]:
ut.df_nodes()

group_varsを更新します。

In [ ]:
%run scripts/group.py
!cp group_vars/{ugroup_name} group_vars/.{ugroup_name}.bak || true
remove_group_vars(ugroup_name, "vcnode_schedule", if_exists=True)
remove_group_vars(ugroup_name, "vcnode_all_ipaddress", if_exists=True)
remove_group_vars(ugroup_name, "schedule_down_type", if_exists=True)
update_group_vars(ugroup_name, worker_nodes=worker_nodes)
update_group_vars(ugroup_name, worker_ipaddresses=ut.find_ip_addresses())